In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
#import yfinance as yf
# import tensorflow as tf
# import tensorflow.compat.v2.feature_column as fc
from IPython.display import clear_output
import pyodbc

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')

from utils.data_extraction import load_data_final, load_data_and_save_v2, strList
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final
from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing
# from utils.models import make_input_fn
# from utils.models import performanceTesting,scoreFunction

### Load data etc

In [3]:
stockInfo = pd.read_csv('../utils/stockInfo.csv',header=[0,1])

In [4]:
stockInfo.dropna()[stockInfo.dropna()[('2020-07-05','exchange')]=='NMS'].sort_values(('2020-07-05','marketCap'))

date              2020-07-05                       
     ticker                  sector exchange     marketCap
7671   TCCO              Technology      NMS  5.902776e+06
2323   EDRY             Industrials      NMS  8.550177e+06
5016   MDRR             Real Estate      NMS  8.736265e+06
4262   ISIG  Communication Services      NMS  9.193270e+06
5644   NSYS              Technology      NMS  1.063012e+07
...     ...                     ...      ...           ...
4181   INTC              Technology      NMS  2.503564e+11
2733     FB  Communication Services      NMS  6.650369e+11
3462   GOOG  Communication Services      NMS  1.001645e+12
5272   MSFT              Technology      NMS  1.564160e+12
12     AAPL              Technology      NMS  1.578173e+12

[1295 rows x 4 columns]

In [5]:
# How many are there contained in the file?
print('There are',
      stockInfo.shape[0],
      'tickers contained in the file, and there is data on',
      stockInfo.isnull().sum()[1:].min(),'of them.')

There are 8850 tickers contained in the file, and there is data on 5306 of them.


## Sneak peak on the 10 largest companies

In [6]:

stockInfo.sort_values(('2020-07-05','marketCap'),ascending=False).head(10)

date              2020-07-05                       
     ticker                  sector exchange     marketCap
12     AAPL              Technology      NMS  1.578173e+12
5272   MSFT              Technology      NMS  1.564160e+12
3462   GOOG  Communication Services      NMS  1.001645e+12
2733     FB  Communication Services      NMS  6.650369e+11
699    BABA       Consumer Cyclical      NYQ  5.936536e+11
8228      V      Financial Services      NYQ  4.298068e+11
4459    JNJ              Healthcare      NYQ  3.716463e+11
8583    WMT      Consumer Defensive      NYQ  3.375968e+11
4930     MA      Financial Services      NYQ  3.035511e+11
7950    TSM              Technology      NYQ  2.837483e+11

## How is the tickers divided in sectors?

In [7]:
stockInfo.loc[:,('2020-07-05')].groupby(['sector']).count()#rename(None,axis=1)

,exchange,marketCap
sector,,
Basic Materials,137,137
Communication Services,146,146
Consumer Cyclical,318,317
Consumer Defensive,130,130
Energy,204,204
Financial,3,3
Financial Services,942,942
Healthcare,607,607
Industrials,404,404


## Lets get the X largest companies in each sector.

In [8]:
sectors = stockInfo.loc[:,('2020-07-05')].dropna(axis=0).sector.unique()

X = 5

topXsectors = pd.DataFrame(index = np.arange(X),columns = pd.MultiIndex.from_product([['Top {}'.format(X)],sectors]))

t1 = pd.DataFrame({'tickers':stockInfo.loc[:,'date'].ticker.values})
t1[stockInfo.loc[:,'2020-07-05'].columns] = stockInfo.loc[:,'2020-07-05']

for i,sector in enumerate(sectors):
    
    tempSec = t1[t1.sector==sector].sort_values('marketCap',
                                                      ascending=False).dropna(axis=0).values.T
    
    Y = len(tempSec[0][0:X]) 

    topXsectors.loc[0:(Y-1),('Top {}'.format(X),sector)] = tempSec[0][0:Y]

In [9]:
topXsectors

Top 5                                                 \
  Healthcare Basic Materials Consumer Defensive Industrials   
0        JNJ             BHP                WMT         UNP   
1        NVS             LIN                 KO          BA   
2        PFE             ECL                PEP         LMT   
3       ABBV             APD               COST         UPS   
4        ABT             SHW                FMX         RTX   

                                                                         \
  Financial Services Technology Consumer Cyclical Real Estate Utilities   
0                  V       AAPL              BABA         AMT         D   
1                 MA       MSFT              TSLA         CCI        SO   
2               PYPL        TSM                TM         PLD       AEP   
3                BAC       INTC               NKE         PSA       EXC   
4                LFC       NVDA               MCD        CSGP       SRE   

                                           
  Communication Services Energy Financial  
0                   GOOG    PTR       IIM  
1                     FB     BP       ZTR  
2                      T    SNP      OTTW  
3                    DIS    ENB       NaN  
4                    CHL    PBR       NaN

In [10]:
topXsectors.columns = topXsectors.columns.droplevel(0)

In [11]:
topXsectors

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,JNJ,BHP,WMT,UNP,V,AAPL,BABA,AMT,D,GOOG,PTR,IIM
1,NVS,LIN,KO,BA,MA,MSFT,TSLA,CCI,SO,FB,BP,ZTR
2,PFE,ECL,PEP,LMT,PYPL,TSM,TM,PLD,AEP,T,SNP,OTTW
3,ABBV,APD,COST,UPS,BAC,INTC,NKE,PSA,EXC,DIS,ENB,NaN
4,ABT,SHW,FMX,RTX,LFC,NVDA,MCD,CSGP,SRE,CHL,PBR,NaN


In [12]:
topXsectors.columns!='Financial'

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False])

In [13]:
# List for data extraction
topXsectors.loc[:,topXsectors.columns != 'Financial'].values.flatten()

array(['JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
       'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
       'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
       'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
       'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
       'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR'],
      dtype=object)

In [14]:
stockInfo.head()

date          2020-07-05                       
  ticker              sector exchange     marketCap
0      A          Healthcare      NYQ  2.738234e+10
1     AA     Basic Materials      NYQ  2.039532e+09
2   AAAU                 NaN      NaN           NaN
3   AACG  Consumer Defensive      NGM  4.157730e+07
4   AADR                 NaN      NaN           NaN

In [15]:
'IYZ' in stockInfo.loc[:,('date','ticker')].values

True

In [16]:
sectors

array(['Healthcare', 'Basic Materials', 'Consumer Defensive',
       'Industrials', 'Financial Services', 'Technology',
       'Consumer Cyclical', 'Real Estate', 'Utilities',
       'Communication Services', 'Energy', 'Financial'], dtype=object)

In [17]:
## IShares Sector ETFS

# iShares Dow Jones U.S. Basic Materials Index:     IYM
# iShares Dow Jones U.S. Consumer Goods Index:      IYK
# iShares Dow Jones U.S. Consumer Services Index:   IYC
# iShares Dow Jones U.S. Energy Index:              IYE
# iShares Dow Jones U.S. Financial Sector Index:    IYF
# iShares Dow Jones U.S. Financial Services Index:  IYG
# iShares Dow Jones U.S. Healthcare Index:          IYH
# iShares Dow Jones U.S. Industrial Index:          IYJ
# iShares Dow Jones U.S. Real Estate Index:         IYR
# iShares Dow Jones U.S. Technology Index:          IYW
# iShares Dow Jones U.S. Telecommunications Index:  IYZ
# iShares Dow Jones Transportation Average Index:   IYT
# iShares Dow Jones U.S. Utilities Index:           IDU
# iShares Cohen & Steers Realty Majors Index:       ICF

etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF']
# {i:j for i,j in zip(sectors,etfs)}
pd.DataFrame({i:j for i,j in zip(sectors,etfs)},index=[0])

,Healthcare,Basic Materials,Consumer Defensive,Industrials,Financial Services,Technology,Consumer Cyclical,Real Estate,Utilities,Communication Services,Energy,Financial
0,IYH,IYM,IYK,IYJ,IYG,IYW,IYC,IYR,IDU,IYZ,IYE,IYF


## Let's extract some data now!

In [18]:
all_tickers = ['JNJ', 'BHP', 'WMT', 'UNP', 'V', 'AAPL', 'BABA', 'AMT', 'D',
               'GOOG', 'PTR', 'NVS', 'LIN', 'KO', 'BA', 'MA', 'MSFT', 'TSLA',
               'CCI', 'SO', 'FB', 'BP', 'PFE', 'ECL', 'PEP', 'LMT', 'PYPL', 'TSM',
               'TM', 'PLD', 'AEP', 'T', 'SNP', 'ABBV', 'APD', 'COST', 'UPS',
               'BAC', 'INTC', 'NKE', 'PSA', 'EXC', 'DIS', 'ENB', 'ABT', 'SHW',
               'FMX', 'RTX', 'LFC', 'NVDA', 'MCD', 'CSGP', 'SRE', 'CHL', 'PBR',
              'IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE'
              ]

In [19]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200501',
                  '20200504',
                  '20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529']).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

exchanges = ['q','t']
select_largest_exchange = True

# run load_data()
candles = load_data_and_save_v2(dates,
                                  tickers,
                                  dataNeeded,
                                  path,
                                  verbose,
                                  extract_candles,
                                  aggHorizon,
                                  extra_features_from_quotes,
                                  data_sample,
                                  exchanges,
                                  save_output,
                                  select_largest_exchange)

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200501
Date, Max: 20200529


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.001

### Quote Data ###

3 Lap time: 0.022
4 Lap time: 0.022
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.874
6 Lap time: 0.884
7 Lap time: 0.900
8 Lap time: 80.013
9 Lap time: 94.945
10 Lap time: 109.927
11 Lap time: 110.299
12 Lap time: 115.006
Step: 0.16666666666666666 -- Ticker: AAPL -- Exchange sizes are: ex
Q

Step: 0.5 -- Ticker: ECL -- Exchange sizes are: ex
T    34451
dtype: int64
Step: 0.5 -- Ticker: ENB -- Exchange sizes are: ex
T    235983
dtype: int64
Step: 0.5 -- Ticker: EXC -- Exchange sizes are: ex
Q    300923
dtype: int64
Step: 0.5 -- Ticker: FB -- Exchange sizes are: ex
Q    294585
dtype: int64
Step: 0.5 -- Ticker: FMX -- Exchange sizes are: ex
T    10526
dtype: int64
Step: 0.5 -- Ticker: GOOG -- Exchange sizes are: ex
Q    135926
dtype: int64
Step: 0.5 -- Ticker: IDU -- Exchange sizes are: ex
T    89486
dtype: int64
Step: 0.5 -- Ticker: INTC -- Exchange sizes are: ex
Q    688595
dtype: int64
Step: 0.5 -- Ticker: IYC -- Exchange sizes are: ex
T    104041
dtype: int64
Step: 0.5 -- Ticker: IYE -- Exchange sizes are: ex
T    149667
dtype: int64
Step: 0.5 -- Ticker: IYG -- Exchange sizes are: ex
T    87168
dtype: int64
Step: 0.5 -- Ticker: IYH -- Exchange sizes are: ex
T    69596
dtype: int64
Step: 0.5 -- Ticker: IYJ -- Exchange sizes are: ex
T    86208
dtype: int64
Step: 0.5 -- Tick

Step: 1 -- Ticker: UNP -- Exchange sizes are: ex
T    96008
dtype: int64
Step: 1 -- Ticker: UPS -- Exchange sizes are: ex
T    79318
dtype: int64
Step: 1 -- Ticker: V -- Exchange sizes are: ex
T    191594
dtype: int64
Step: 1 -- Ticker: WMT -- Exchange sizes are: ex
T    177821
dtype: int64
3 Lap time: 438.907
4 Lap time: 438.908
5 Lap time: 439.782
6 Lap time: 439.793
7 Lap time: 439.901
8 Lap time: 516.625
9 Lap time: 531.814
10 Lap time: 547.021
11 Lap time: 547.400
12 Lap time: 552.267
Step: 0.16666666666666666 -- Ticker: AAPL -- Exchange sizes are: ex
Q    682135
dtype: int64
Step: 0.16666666666666666 -- Ticker: ABBV -- Exchange sizes are: ex
T    162315
dtype: int64
Step: 0.16666666666666666 -- Ticker: ABT -- Exchange sizes are: ex
T    97886
dtype: int64
Step: 0.16666666666666666 -- Ticker: AEP -- Exchange sizes are: ex
T    70721
dtype: int64
Step: 0.16666666666666666 -- Ticker: AMT -- Exchange sizes are: ex
T    31127
dtype: int64
Step: 0.16666666666666666 -- Ticker: APD -- Ex

Step: 0.5 -- Ticker: IDU -- Exchange sizes are: ex
T    64475
dtype: int64
Step: 0.5 -- Ticker: INTC -- Exchange sizes are: ex
Q    694560
dtype: int64
Step: 0.5 -- Ticker: IYC -- Exchange sizes are: ex
T    86331
dtype: int64
Step: 0.5 -- Ticker: IYE -- Exchange sizes are: ex
T    191452
dtype: int64
Step: 0.5 -- Ticker: IYG -- Exchange sizes are: ex
T    84318
dtype: int64
Step: 0.5 -- Ticker: IYH -- Exchange sizes are: ex
T    68859
dtype: int64
Step: 0.5 -- Ticker: IYJ -- Exchange sizes are: ex
T    53878
dtype: int64
Step: 0.5 -- Ticker: IYK -- Exchange sizes are: ex
T    42007
dtype: int64
Step: 0.5 -- Ticker: IYM -- Exchange sizes are: ex
T    79218
dtype: int64
Step: 0.5 -- Ticker: IYR -- Exchange sizes are: ex
T    432936
dtype: int64
Step: 0.5 -- Ticker: IYW -- Exchange sizes are: ex
T    218840
dtype: int64
Step: 0.5 -- Ticker: IYZ -- Exchange sizes are: ex
T    31926
dtype: int64
Step: 0.5 -- Ticker: JNJ -- Exchange sizes are: ex
T    118536
dtype: int64
Step: 0.5 -- Ticker

9 Lap time: 998.808
10 Lap time: 1013.500
11 Lap time: 1013.869
12 Lap time: 1018.567
Step: 0.16666666666666666 -- Ticker: AAPL -- Exchange sizes are: ex
Q    729010
dtype: int64
Step: 0.16666666666666666 -- Ticker: ABBV -- Exchange sizes are: ex
T    225423
dtype: int64
Step: 0.16666666666666666 -- Ticker: ABT -- Exchange sizes are: ex
T    119726
dtype: int64
Step: 0.16666666666666666 -- Ticker: AEP -- Exchange sizes are: ex
T    70788
dtype: int64
Step: 0.16666666666666666 -- Ticker: AMT -- Exchange sizes are: ex
T    40230
dtype: int64
Step: 0.16666666666666666 -- Ticker: APD -- Exchange sizes are: ex
T    30799
dtype: int64
Step: 0.16666666666666666 -- Ticker: BA -- Exchange sizes are: ex
T    221050
dtype: int64
Step: 0.16666666666666666 -- Ticker: BABA -- Exchange sizes are: ex
T    261463
dtype: int64
Step: 0.16666666666666666 -- Ticker: BAC -- Exchange sizes are: ex
T    449675
dtype: int64
Step: 0.16666666666666666 -- Ticker: BHP -- Exchange sizes are: ex
T    181240
dtype: i

...

Step: 1 -- Ticker: LIN -- Exchange sizes are: ex
T    45529
dtype: int64
Step: 1 -- Ticker: LMT -- Exchange sizes are: ex
T    50463
dtype: int64
Step: 1 -- Ticker: MA -- Exchange sizes are: ex
T    170125
dtype: int64
Step: 1 -- Ticker: MCD -- Exchange sizes are: ex
T    140614
dtype: int64
Step: 1 -- Ticker: MSFT -- Exchange sizes are: ex
Q    1316370
dtype: int64
Step: 1 -- Ticker: NKE -- Exchange sizes are: ex
T    172301
dtype: int64
Step: 1 -- Ticker: NVDA -- Exchange sizes are: ex
Q    326891
dtype: int64
Step: 1 -- Ticker: NVS -- Exchange sizes are: ex
T    60249
dtype: int64
Step: 1 -- Ticker: PBR -- Exchange sizes are: ex
T    257392
dtype: int64
Step: 1 -- Ticker: PEP -- Exchange sizes are: ex
Q    156977
dtype: int64
Step: 1 -- Ticker: PFE -- Exchange sizes are: ex
T    463159
dtype: int64
Step: 1 -- Ticker: PLD -- Exchange sizes are: ex
T    67744
dtype: int64
Step: 1 -- Ticker: PSA -- Exchange sizes are: ex
T    28628
dtype: int64
Step: 1 -- Ticker: PTR -- Exchange sizes 

Step: 0.16666666666666666 -- Ticker: WMT -- Exchange sizes are: ex
T    205244
dtype: int64
Step: 0.5 -- Ticker: AAPL -- Exchange sizes are: ex
Q    708173
dtype: int64
Step: 0.5 -- Ticker: ABBV -- Exchange sizes are: ex
T    190442
dtype: int64
Step: 0.5 -- Ticker: ABT -- Exchange sizes are: ex
T    154076
dtype: int64
Step: 0.5 -- Ticker: AEP -- Exchange sizes are: ex
T    81300
dtype: int64
Step: 0.5 -- Ticker: AMT -- Exchange sizes are: ex
T    44379
dtype: int64
Step: 0.5 -- Ticker: APD -- Exchange sizes are: ex
T    34267
dtype: int64
Step: 0.5 -- Ticker: BA -- Exchange sizes are: ex
T    366984
dtype: int64
Step: 0.5 -- Ticker: BABA -- Exchange sizes are: ex
T    146337
dtype: int64
Step: 0.5 -- Ticker: BAC -- Exchange sizes are: ex
T    825904
dtype: int64
Step: 0.5 -- Ticker: BHP -- Exchange sizes are: ex
T    274728
dtype: int64
Step: 0.5 -- Ticker: BP -- Exchange sizes are: ex
T    399303
dtype: int64
Step: 0.5 -- Ticker: CCI -- Exchange sizes are: ex
T    53063
dtype: int64

Step: 1 -- Ticker: NKE -- Exchange sizes are: ex
T    207559
dtype: int64
Step: 1 -- Ticker: NVDA -- Exchange sizes are: ex
Q    247391
dtype: int64
Step: 1 -- Ticker: NVS -- Exchange sizes are: ex
T    60567
dtype: int64
Step: 1 -- Ticker: PBR -- Exchange sizes are: ex
T    229568
dtype: int64
Step: 1 -- Ticker: PEP -- Exchange sizes are: ex
Q    208953
dtype: int64
Step: 1 -- Ticker: PFE -- Exchange sizes are: ex
T    407510
dtype: int64
Step: 1 -- Ticker: PLD -- Exchange sizes are: ex
T    60331
dtype: int64
Step: 1 -- Ticker: PSA -- Exchange sizes are: ex
T    28737
dtype: int64
Step: 1 -- Ticker: PTR -- Exchange sizes are: ex
T    7348
dtype: int64
Step: 1 -- Ticker: PYPL -- Exchange sizes are: ex
Q    268384
dtype: int64
Step: 1 -- Ticker: RTX -- Exchange sizes are: ex
T    215219
dtype: int64
Step: 1 -- Ticker: SHW -- Exchange sizes are: ex
T    17666
dtype: int64
Step: 1 -- Ticker: SNP -- Exchange sizes are: ex
T    9152
dtype: int64
Step: 1 -- Ticker: SO -- Exchange sizes are:

Step: 0.5 -- Ticker: AMT -- Exchange sizes are: ex
T    48108
dtype: int64
Step: 0.5 -- Ticker: APD -- Exchange sizes are: ex
T    53034
dtype: int64
Step: 0.5 -- Ticker: BA -- Exchange sizes are: ex
T    388029
dtype: int64
Step: 0.5 -- Ticker: BABA -- Exchange sizes are: ex
T    202255
dtype: int64
Step: 0.5 -- Ticker: BAC -- Exchange sizes are: ex
T    871133
dtype: int64
Step: 0.5 -- Ticker: BHP -- Exchange sizes are: ex
T    452268
dtype: int64
Step: 0.5 -- Ticker: BP -- Exchange sizes are: ex
T    444353
dtype: int64
Step: 0.5 -- Ticker: CCI -- Exchange sizes are: ex
T    56847
dtype: int64
Step: 0.5 -- Ticker: CHL -- Exchange sizes are: ex
T    69714
dtype: int64
Step: 0.5 -- Ticker: COST -- Exchange sizes are: ex
Q    98340
dtype: int64
Step: 0.5 -- Ticker: CSGP -- Exchange sizes are: ex
Q    18919
dtype: int64
Step: 0.5 -- Ticker: D -- Exchange sizes are: ex
T    102227
dtype: int64
Step: 0.5 -- Ticker: DIS -- Exchange sizes are: ex
T    236587
dtype: int64
Step: 0.5 -- Ticker

Step: 1 -- Ticker: PFE -- Exchange sizes are: ex
T    578441
dtype: int64
Step: 1 -- Ticker: PLD -- Exchange sizes are: ex
T    68024
dtype: int64
Step: 1 -- Ticker: PSA -- Exchange sizes are: ex
T    32976
dtype: int64
Step: 1 -- Ticker: PTR -- Exchange sizes are: ex
T    8157
dtype: int64
Step: 1 -- Ticker: PYPL -- Exchange sizes are: ex
Q    288483
dtype: int64
Step: 1 -- Ticker: RTX -- Exchange sizes are: ex
T    117250
dtype: int64
Step: 1 -- Ticker: SHW -- Exchange sizes are: ex
T    21309
dtype: int64
Step: 1 -- Ticker: SNP -- Exchange sizes are: ex
T    9516
dtype: int64
Step: 1 -- Ticker: SO -- Exchange sizes are: ex
T    216574
dtype: int64
Step: 1 -- Ticker: SRE -- Exchange sizes are: ex
T    48132
dtype: int64
Step: 1 -- Ticker: T -- Exchange sizes are: ex
T    589506
dtype: int64
Step: 1 -- Ticker: TM -- Exchange sizes are: ex
T    23175
dtype: int64
Step: 1 -- Ticker: TSLA -- Exchange sizes are: ex
Q    167810
dtype: int64
Step: 1 -- Ticker: TSM -- Exchange sizes are: ex


In [21]:
# print(os.listdir())
try:
    path = 'a:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)
except:
    path = 'f:/taqhdf5'  #'a:/taqhdf5'
    os.listdir(path)    
# allFiles = os.listdir(path)
# print(len(allFiles), allFiles[:5], allFiles[-5:])
# print(allFiles[-10:])

#dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
dates = np.array(['20200505',
                  '20200506',
                  '20200507',
                  '20200508',
                  '20200511',
                  '20200512',
                  '20200513',
                  '20200514',
                  '20200515',
                  '20200518',
                  '20200519',
                  '20200520',
                  '20200521',
                  '20200522',
                  '20200526',
                  '20200527',
                  '20200528',
                  '20200529'
                  
                  
                 ]).astype(int)#,'20200401','20200402','20200403','20200406','20200407'

# Provide a list of tickers of interest
tickers = sorted(all_tickers) # ['GOOG']#'MSFT' ['GOOG', 'MSFT'] #

# Do we need data on trades, quotes or both?
dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'

extract_candles = True #False
aggHorizon = [1/6, 1/2, 1] # in minutes
extra_features_from_quotes = ['spread', 'bidsize', 'ofrsize']

data_sample = 'full'
save_output = True

# run load_data()
candles = load_data_final(dates,
                          tickers,
                          dataNeeded,
                          path,
                          verbose,
                          extract_candles,
                          aggHorizon,
                          extra_features_from_quotes,
                          data_sample,
                          save_output)

dates,
                    tickers,
                    dataNeeded,
                    path,
                    verbose,
                    extract_candles = False,
                    aggHorizon = 1,
                    extra_features_from_quotes = None,
                    data_sample = 'full',
                    exchanges = ['q','t'],
                    save_output = False):

# quoteData = load_data_final(dates,
#                           tickers,
#                           dataNeeded,
#                           path,
#                           verbose,
#                           extract_candles,
#                           aggHorizon,
#                           extra_features_from_quotes,
#                           data_sample,
#                           save_output)

# if dataNeeded == 'trades':
#     tradeData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'quotes':
#     quoteData = load_data(dates, tickers, dataNeeded, path, verbose)
# elif dataNeeded == 'both':
#     tradeData, quoteData = load_data(dates, tickers, dataNeeded, path, verbose)

20 ['taqquote_20200501.h5', 'taqquote_20200504.h5', 'taqquote_20200505.h5', 'taqquote_20200506.h5', 'taqquote_20200507.h5'] ['taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
['taqquote_20200515.h5', 'taqquote_20200518.h5', 'taqquote_20200519.h5', 'taqquote_20200520.h5', 'taqquote_20200521.h5', 'taqquote_20200522.h5', 'taqquote_20200526.h5', 'taqquote_20200527.h5', 'taqquote_20200528.h5', 'taqquote_20200529.h5']
##### Date range #####

Date, Min: 20200505
Date, Max: 20200529


1 Lap time: 0.001

##### Data Extraction begins #####

quote data is being extracted..


2 Lap time: 0.002

### Quote Data ###

3 Lap time: 0.076
4 Lap time: 0.076
The raw H5 quote file contains:  ['QuoteIndex', 'Quotes'] 

5 Lap time: 0.868
6 Lap time: 0.878
7 Lap time: 62.353
8 Lap time: 77.898
9 Lap time: 92.528
10 Lap time: 107.148
11 Lap time: 107.512
12 Lap time: 112.094
3 Lap time: 353.492
4 Lap time: 353.493
5 Lap time: 354.299
6 Lap ti

TypeError: unsupported operand type(s) for /: 'list' and 'int'